In [2]:
import sys
import os
import ctypes
import copy


# Get the absolute path of the utils directory
utils_path = os.path.abspath("../lib")
sys.path.append(utils_path)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import qiskit.qpy as qpy
import pickle
import json
import matplotlib.patches as mpatches
from numba import njit
from numba.typed import List
from plotting import *
from readings import *
from tile_process import *
import random

In [ ]:
seam_lst = [8,16]
f_orbs = (2,2,2,2,2,2)
read_epsilon = 1.0
distance = 3.0
epsilon_lst = np.linspace(3.1,3.9,9)
epsilon_lst = 10**-(epsilon_lst)
ratio_lst = np.linspace(1, 15, 15)
# ratio_lst = np.concatenate((ratio_lst1, ratio_lst))

path = f"../../H_12_transition/prepacked/inter_transition_{distance+1}/enhanced_epsilon = {epsilon}.npy"
directory = os.path.dirname(path)
os.makedirs(directory, exist_ok=True)

for epsilon in epsilon_lst:
    print(f"cur epsilon is {epsilon}")
    prehead = "../../H_12_cluster/"
    uop, all_g = read_uop(distance, read_epsilon, prehead)
    # inter_single, intra_single, inter_control, intra_control, inter_double, intra_double = plot_uop(uop, all_g, 1e-5, seam_lst,epsilon_lst, f_orbs, total=True)
    excitations = create_excitation(uop, all_g, epsilon)
    excitations = orbital_reordering(excitations, f_orbs)
    print(f"excitation has length {len(excitations)}")
    post_gate_lst = np.zeros(len(ratio_lst))
    print(len(ratio_lst))
    for i in range(len(ratio_lst)):
        print(f"current ratio is {ratio_lst[i]}")
        separation = int(2*(ratio_lst[i] - 1))
        print(f"current separation is {separation}")
        bounding_width, placed_tiles_lst = preplace_pack_with_c(excitations, separation, seam_lst)
        print(f"C bounding width is {bounding_width}")
        post_gate_lst[i] = bounding_width * 25
    post_gate_lst = np.array(post_gate_lst)
    initial_gate = post_gate_lst[0]
    diff = post_gate_lst - initial_gate
    plt.plot(ratio_lst, diff/initial_gate)
    plt.xlabel("inter/intra")
    plt.ylabel("increase of time(%)")
    post_gate_lst.tolist()
    ratio_lst.tolist()
    initial_gate.tolist()
    result = [ratio_lst, post_gate_lst]
    
    np.save(f"../../H_12_transition/double_packed/inter_transition_{distance+1}/enhanced_epsilon = {epsilon}.npy", result)